# 딥러닝

## 4. 딥러닝 활용 예제

### [실습] Tensorflow-Keras를 이용한 당뇨 예측 신경망 모델 만들기

- **신경망 모델 훈련 단계**
1. 학습 데이터 준비
2. 신경망 모델 정의
3. 컴파일 과정 설정
4. 모델 훈련
5. 예측 및 평가

#### 1.학습 데이터 준비
피마 인디언 당뇨병 데이터(diabetes.csv)
- https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
- 21세 이상의 피마 인디언 혈통 여성 데이터
(all patients here are females at least 21 years old of Pima Indian heritage)
> - Pregnancies: 과거 임신 횟수
> - Glucose: 혈당 수치
> - BloodPressure: 혈압
> - SkinThickness: 피부겹두께
> - Insulin: 인슐린 농도
> - BMI: 체질량 지수(Body Mass Index) 
> - DiabetesPedigreeFunction: 가족력을 바탕으로 환자가 당뇨에 얼마나 취약한지를 요약한 점수
> - Age: 나이
> - **Outcome**: (레이블) 예측 목표 변수(최초 측정 후 5년 내 당뇨 발병=1, 미발병=0)


In [ ]:
import pandas as pd

df = pd.read_csv('data/diabetes.csv')
df.head()

##### 컬럼 히스토그램으로 시각화

In [ ]:
import matplotlib.pyplot as plt

df.hist()
plt.tight_layout() # 서브 플롯의 크기/간격 일정하게
plt.show()

##### 결측치 처리
- 0인 데이터 --> NaN--> 평균값으로 대체
- replace(0, np.nan) --> fillna(df[컬럼].mean())

In [ ]:
df.describe()

- **1단계**: 0인 데이터 --> NaN : replace(0, np.nan) 

In [ ]:
import numpy as np

cols = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
for col in cols:
    df[col] = df[col].replace(0, np.nan)
    
df.describe()

- **2단계**: NaN 데이터 --> fillna(df[컬럼].mean())

In [ ]:
for col in cols:
    df[col] = df[col].fillna(df[col].mean())
    
df.describe()

##### 데이터 표준화
- 범위가 다른 각 컬럼의 수치 데이터들을 평균 0, 분산 1이 되도록 변환
- 사이킷런 preprocessing 사용

In [ ]:
from sklearn import preprocessing

df_scaled = preprocessing.scale(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
df_scaled
df_scaled['Outcome'] = df['Outcome']
df = df_scaled
df

df.describe().loc[['min','mean','max','std']].round(2)

##### 데이터셋 분할
- 사이킷런 train_test_split 이용해서 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

X = df.loc[:, df.columns!='Outcome']
Y = df.loc[:, 'Outcome']

x_train, x_test, y_train, y_test =  train_test_split(X, Y, test_size=0.2)
print(f'x_train, x_test : {x_train.shape, x_test.shape}')
print(f'y_train, y_test : {y_train.shape, y_test.shape}')

#### 2.신경망 모델 정의

In [ ]:
# 방법1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()  
model.add(Dense(32, activation='relu', input_dim=8)) # 첫 번째 은닉층
model.add(Dense(16, activation='relu')) # 두 번째 은닉층
model.add(Dense(1, activation='sigmoid')) # 출력 레이어


# 방법2
model = Sequential([
    Dense(32, activation='relu', input_dim=8),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')          
    ])

#### 3.컴파일 과정 설정

In [ ]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

#### 4.모델 훈련

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=10)

#### 5.예측 및 평가

In [ ]:
loss, accuracy = model.evaluate(x_train, y_train)
print(f'Training Accuracy: {accuracy*100:>.2f}%')

loss, accuracy = model.evaluate(x_test, y_test)
print(f'Testing Accuracy: {accuracy*100:>.2f}%')

In [ ]:
# 예측해보기
#8	183	64	0	0	23.3	0.672	32	1
#1	89	66	23	94	28.1	0.167	21	0
# test = np.array([[8,183,64,0,0,23.3,0.672,32]]) 
# test = np.array([[1,89,66,23,94,28.1,0.167,21]]) 
model.predict(test)

In [ ]:
model.summary()

-----

### [실습] 보스턴 집값 예측모델 만들기

In [ ]:
#---------------------------
# 1. 학습 데이터 준비
#---------------------------
import pandas as pd

df = pd.read_csv('data/HousingData.csv')
df.head()

# 컬럼 히스토그램으로 시각화
# import matplotlib.pyplot as plt
# df.hist()
# plt.tight_layout() # 서브 플롯의 크기/간격 일정하게
# plt.show()

# 데이터 표준화
from sklearn import preprocessing

df_scaled = preprocessing.scale(df)
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
df_scaled
df_scaled['MEDV'] = df['MEDV']
df = df_scaled
df

df.describe().loc[['min','mean','max','std']].round(2)


# 데이터셋 분할
from sklearn.model_selection import train_test_split

X = df.loc[:, df.columns!='MEDV']
Y = df.loc[:, 'MEDV']

x_train, x_test, y_train, y_test =  train_test_split(X, Y, test_size=0.2)
print(f'x_train, x_test : {x_train.shape, x_test.shape}')
print(f'y_train, y_test : {y_train.shape, y_test.shape}')
x_train


In [ ]:
# 1. 학습 데이터 준비
from tensorflow.keras.datasets import boston_housing   

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

print(type(x_train))
print(x_train.shape, x_test.shape)
print(x_train[0])
print(y_train[0])


# 데이터 정규화
# 독립변수(Feature에 대한 데이터 정규화 진행)
# 특성의 중앙이 0 근처에 맞추어지고 표준 편차를 1이 되도록한다.
x_mean = x_train.mean(axis=0)
x_std = x_train.std(axis=0)
x_train -= x_mean
x_train /= x_std
x_test -= x_mean
x_test /= x_std
x_train


import matplotlib.pyplot as plt
plt.hist(x_train)      
plt.show()

In [ ]:
#---------------------------
# 2. 신경망 모델 정의
#---------------------------
# 방법1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()  
model.add(Dense(64, activation='relu', input_dim=13)) # 첫 번째 은닉층
model.add(Dense(64, activation='relu')) # 두 번째 은닉층
model.add(Dense(1)) # 출력 레이어

#---------------------------
# 3. 컴파일 과정 설정
#---------------------------
model.compile(optimizer='rmsprop',
             loss='mse',
             metrics=['mae'])

#---------------------------
# 4. 모델 훈련
#---------------------------
model.fit(x_train, y_train, epochs=100, batch_size=10, verbose=0)

#---------------------------
# 5. 예측 및 평가
#---------------------------
mse_score, mae_score = model.evaluate(x_test, y_test)
print(f'Testing mse_score: {mse_score:>.2f}%')
print(f'Testing mae_score: {mae_score:>.2f}%')

------

### [실습] Teachable Machine(Tensorflow) 모델 이용하여 마스크 착용 판별여부 판단하기

- 모델생성 : Teachable Machine(Tensorflow)
- **구현할 부분**:  실시간으로 카메라로 마스크 착용여부 판단하는 부분

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import sys
import win32com.client

# 케라스 모델 가져오기
model = tf.keras.models.load_model('converted_keras/keras_model.h5')


#-------------------
# 실시간 카메라 작동
#-------------------
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

CONFIDENCE = 0.8
while True:
    ret, frame = cap.read()
    if not ret:
        print('Video read failed!')
        break
    
    #-------------------
    # 마스크 착용 판별하기 ---
    # 모델에서 원하는 사이즈로 resizing
    size = (224, 224)
    frame_resized = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)

    # 이미지 정규화            
    frame_normalized = (frame_resized.astype(np.float32) / 127.0) - 1
    
    # keras에서 원하는 모양(차원으로 변환)
    frame_reshape = frame_normalized.reshape(1, 224, 224, 3)
    
    # 예측결과
    prediction = model.predict(frame_reshape)
#     print(prediction.shape)
#     print(prediction)
    
    # prediction: [[0.13399076 0.86600924]]
    if prediction[0, 0] > CONFIDENCE : # 0(mask)일때 정확도
        cv2.putText(frame, 'Mask', (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,0), 2, cv2.LINE_AA) # 검정
#         print('mask')
    elif prediction[0, 1] > CONFIDENCE : # 2(nmask)일때 정확도
        cv2.putText(frame, 'Nomask', (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,255), 2, cv2.LINE_AA) # 빨강
#         print('nomask')
    
    #-------------------
            
    cv2.imshow('Mask Detect', frame)
    
    key = cv2.waitKey(1)
    if key==27: # ESC키 눌러졌을 때 프로그램 나가기
        break
    elif key==13:
        cv2.imwrite('image/photo.jpg', frame)

    
cap.release()
cv2.destroyAllWindows()
print('Video Finish!')

#### TTS 기능 추가하기

In [ ]:
# 윈도우에서 tts사용 라이브러리 설치
!pip install pywin32

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import sys
import win32com.client

# win TTS 사용
tts = win32com.client.Dispatch("SAPI.SpVoice")

# 케라스 모델 가져오기
model = tf.keras.models.load_model('converted_keras/keras_model.h5')


#-------------------
# 실시간 카메라 작동
#-------------------
tts.Speak("카메라가 실행됩니다.")  

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Video open failed!')
    sys.exit()

CONFIDENCE = 0.8
while True:
    ret, frame = cap.read()
    if not ret:
        print('Video read failed!')
        break
    
    #-------------------
    # 마스크 착용 판별하기 ---
    # 모델에서 원하는 사이즈로 resizing
    size = (224, 224)
    frame_resized = cv2.resize(frame, size, interpolation=cv2.INTER_AREA)

    # 이미지 정규화            
    frame_normalized = (frame_resized.astype(np.float32) / 127.0) - 1
    
    # keras에서 원하는 모양(차원으로 변환)
    frame_reshape = frame_normalized.reshape(1, 224, 224, 3)
    
    # 예측결과
    prediction = model.predict(frame_reshape)
#     print(prediction.shape)
#     print(prediction)
    
    # prediction: [[0.13399076 0.86600924]]
    if prediction[0, 0] > CONFIDENCE : # 0(mask)일때 정확도
        cv2.putText(frame, 'Mask', (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,0), 2, cv2.LINE_AA) # 검정
#         print('mask')
    elif prediction[0, 1] > CONFIDENCE : # 2(nmask)일때 정확도
        cv2.putText(frame, 'Nomask', (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0,0,255), 2, cv2.LINE_AA) # 빨강
#         print('nomask')
        tts.Speak('마스크를 착용하세요!')
    
    #-------------------
            
    cv2.imshow('Mask Detect', frame)
    
    key = cv2.waitKey(1)
    if key==27: # ESC키 눌러졌을 때 프로그램 나가기
        break


    
cap.release()
cv2.destroyAllWindows()
print('Video Finish!')

-------

### [실습] 마우스로 직접 쓴 숫자 인식하는 프로그램 만들기

#### # Tensorflow MNIST 모델 만들기

In [ ]:
import numpy as np
import cv2
import sys

oldx, oldy = -1, -1

# -------------------
# 마우스 이벤트
# -------------------
def on_mouse(event, x, y, flags, _):
    global oldx, oldy

    if event == cv2.EVENT_LBUTTONDOWN:
        oldx, oldy = x, y

    elif event == cv2.EVENT_LBUTTONUP:
        oldx, oldy = -1, -1

    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.line(img, (oldx, oldy), (x, y), (255, 255, 255), 40, cv2.LINE_AA)
            oldx, oldy = x, y
            cv2.imshow('img', img)
                        
# -------------------
# 1. 학습모델 만들기
# -------------------
# Tensorflow MNIST 모델 만들기
# 1.학습 데이터 준비
import tensorflow as tf
mnist = tf.keras.datasets.mnist

# MNIST 데이터 로딩하기
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

# 2.신경망 모델 정의
# 방법1
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

# 3.컴파일 과정 설정
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 4.모델 훈련
model.fit(x_train, y_train, epochs=5, verbose=1)

# 5.예측 및 평가
model.evaluate(x_test, y_test)
           
    
    
# -------------------
# 2. 입력 창 만들어 숫자 인식하기
# -------------------
# 사용자 입력 영상에 대해 예측
img = np.zeros((400, 400), np.uint8)

cv2.imshow('img', img)
cv2.setMouseCallback('img', on_mouse)


while True:
    key = cv2.waitKey()
    if key == 27:   # ESC 키
        break
    elif key == 13 : # 엔터 키
        # Tensorflow 모델에 넣어서 예측하기
        test_image = cv2.resize(img, (28, 28), interpolation=cv2.INTER_AREA)
        test_image = test_image.reshape(1, 28, 28)   # np.expand_dims(test_image, axis=0)
        prediction = model.predict(test_image)  
        print(f'prediction : {prediction}')
        print(f'Tensorflow 모델로 인식된 숫자는: {np.argmax(prediction[0])}')
        
        # 화면 clear기능이 되도록 구현
        img.fill(0) # 검은색으로 채우기 -->clear기능이 되도록 구현
        cv2.imshow('img', img)
        
cv2.destroyAllWindows()        

#### OpenCV KNN MNIST 숫자인식 모델

In [ ]:
import numpy as np
import cv2
import sys

oldx, oldy = -1, -1

# -------------------
# 마우스 이벤트
# -------------------
def on_mouse(event, x, y, flags, _):
    global oldx, oldy

    if event == cv2.EVENT_LBUTTONDOWN:
        oldx, oldy = x, y

    elif event == cv2.EVENT_LBUTTONUP:
        oldx, oldy = -1, -1

    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.line(img, (oldx, oldy), (x, y), (255, 255, 255), 40, cv2.LINE_AA)
            oldx, oldy = x, y
            cv2.imshow('img', img)
                        
# -------------------
# 1. 학습모델 만들기
# -------------------
# OpenCV KNN MNIST 모델 만들기
# 이미지를 직접 쪼개서 학습용 데이터와 레이블을 만들고 KNN 알고리즘에 넣어 ML 모델 만들기
digits = cv2.imread('image/digits.png', cv2.IMREAD_GRAYSCALE)
h, w = digits.shape
print(f'digits.shape: {digits.shape}')
print(f'h, w: {h, w}')

# 학습용 이미지크기로 조정하기(20x20)
cells = [np.hsplit(row, w//20)   for row in np.vsplit(digits, h//20)]
cells = np.array(cells)
print(f'cells.shape : {cells.shape}')

# 학습용 데이터 & 레이블 만들기(지도학습 데이터 만들기)
train_images = cells.reshape(-1, 400).astype(np.float32)
train_labels = np.repeat(np.arange(10), len(train_images)/10)
print(f'train_images : {train_images.shape}') # 0~9이미지가 각각 500개씩
print(f'train_labels : {train_labels}') # 0~9이미지의 레이블값

# OpenCV KNN 알고리즘 넣어 학습시키기
knn_model = cv2.ml.KNearest_create()
knn_model.train(train_images, cv2.ml.ROW_SAMPLE, train_labels )
    
    
# -------------------
# 2. 입력 창 만들어 숫자 인식하기
# -------------------
# 사용자 입력 영상에 대해 예측
img = np.zeros((400, 400), np.uint8)

cv2.imshow('img', img)
cv2.setMouseCallback('img', on_mouse)


while True:
    key = cv2.waitKey()
    if key == 27:   # ESC 키
        break
    elif key == 13 : # 엔터 키
        # OpenCV KNN모델에 넣어서 예측하기
        test_image = cv2.resize(img, (20, 20), interpolation=cv2.INTER_AREA)
        test_image = test_image.reshape(-1, 400).astype(np.float32)
        prediction = knn_model.findNearest(test_image, 5)   # k=5
#         print(f'prediction : {prediction}')
        print(f'OpenCV KNN 모델로 인식된 숫자는: {int(prediction[0])}')
        
        # 화면 clear기능이 되도록 구현
        img.fill(0) # 검은색으로 채우기 -->clear기능이 되도록 구현
        cv2.imshow('img', img)
        
cv2.destroyAllWindows()  

------

### [실습] OpenCV DNN을 이용한 얼굴 인식

In [4]:
import cv2
import sys
import numpy as np

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('Camera open failed!')
    sys.exit()

# 얼굴인식용 DNN Caffe모델 가져오기
# model = 'face_detector/res10_300x300_ssd_iter_140000_fp16.caffemodel'
# config= 'face_detector/deploy.prototxt'
model  = 'face_detector/opencv_face_detector_uint8.pb'
config = 'face_detector/opencv_face_detector.pbtxt'

net = cv2.dnn.readNet(model, config)
if net.empty():
    print('readNet failed!')
    sys.exit()

# 실시간으로 얼굴 인식하기    
while True:
    ret, frame = cap.read()
    if not ret:
        print('Camera read failed!')
        break
    
    # DNN에 맞는 얼굴인식용 이미지로 변환하기
    frame_blob = cv2.dnn.blobFromImage(frame, 1, (300,300), (104, 177, 123))
    net.setInput(frame_blob)    
    detect = net.forward()
    detect = detect[0, 0, :, :]
        
    # 인식된 얼굴에 사각형 그리기
    h, w, _ = frame.shape
    for i in range(detect.shape[0]):
#         print(detect)
        
        confidence = detect[i, 2]
        if confidence < 0.5:
            break
        
        # 얼굴의 위치(x,y)좌표
        x1 = int(detect[i,3] * w)
        y1 = int(detect[i,4] * h)
        x2 = int(detect[i,5] * w)
        y2 = int(detect[i,6] * h)
        
        # 얼굴에 박스 그리기
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
        
        # 정확도 숫자 글씨
        title = f'Face: {confidence:4.2f}'
        cv2.putText(frame, title, (x1, y1-1), cv2.FONT_HERSHEY_SIMPLEX, 1, 
                    (0,255,255), 2, cv2.LINE_AA)
        
    
    cv2.imshow('frame', frame)
    
    key = cv2.waitKey(1)
    if key==27: # ESC키 눌러졌을 때 프로그램 나가기
        break


    
cap.release()
cv2.destroyAllWindows()
print('Camera Finish!')    

Camera Finish!


--------

### [실습] YOLO를 이용한 사물인식(이미지 파일 이용)

In [5]:
import cv2

# img = cv2.imread('image/dog.jpg')
img = cv2.imread('image/fruits.jpg')

# 모델 & 설정 파일
model  = 'yolo_v4/yolov4.weights'
config = 'yolo_v4/yolov4.cfg'
labels = 'yolo_v4/coco.names' # 'yolo_v4/voc.names'


# 1.객체 라벨 가져오기
with open(labels, 'r') as f:
    classes = f.read().splitlines()
# print(f'라벨데이터: {len(classes)}, {classes}')
    
# 2.OpenCV DNN 클래스 사용    
net = cv2.dnn.readNetFromDarknet(config, model) 
model = cv2.dnn_DetectionModel(net)
model.setInputParams(scale=1/255, size=(416, 416), swapRB=True)


# 3.객체 검출하면 분류ID, 정확도, 박스 값 리턴하기 
# opencv 4.5.4에서 오류
# detection 으로 인정할 최소정확도 지정(confThreshold=0.6, nmsThreshold=0.4)
classIds, scores, boxes = model.detect(img, confThreshold=0.6, nmsThreshold=0.4)
print(classIds, classes[classIds[0][0]] )
print(f'정확도:\n{scores}')
print(f'박스위치:n{boxes}')


# 4.검출된 객체에 박스, 객체명, 정확도 그리기
for (classId, score, box) in zip(classIds, scores, boxes):
    cv2.rectangle(img, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]),
                  color=(0, 255, 0), thickness=2)
 
    text = '%s: %.2f' % (classes[classId[0]], score)
    cv2.putText(img, text, (box[0], box[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                color=(0, 255, 0), thickness=2)

# 5 화면에 출력하기
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[41]
 [46]
 [47]] cup
정확도:
[[0.9972165]
 [0.9082982]
 [0.8414951]]
박스위치:n[[667  33 200 586]
 [ 94 357 218 283]
 [372 443 176 178]]


### [실습] YOLO를 이용한 실시간 사물인식

In [7]:
import cv2

# -----------------------
# 1.YOLO 사용 준비
# -----------------------
# 모델 & 설정 파일
config = 'yolo_v4/yolov4.cfg'
model  = 'yolo_v4/yolov4.weights'
labels = 'yolo_v4/coco.names'

# 객체 라벨 가져오기
with open(labels, 'r') as f:
    classes = f.read().splitlines()
    
# penCV DNN 클래스 사용    
net = cv2.dnn.readNetFromDarknet(config, model) 
model = cv2.dnn_DetectionModel(net)
model.setInputParams(scale=1/255, size=(416, 416), swapRB=True)

# -----------------------
# 2.카메라 사용 준비
# -----------------------
cap = cv2.VideoCapture(0)

if not cap.isOpened:
    print('--camera open failed!')
    sys.exit()

# -----------------------
# 4.YOLO를 이용하여 frame에서 객체 검출
# -----------------------
# YOLO Object검출 및 출력
def detectAndDisplay(frame):
    
    # 3.객체 검출하면 분류ID, 정확도, 박스 값 리턴하기 
    # detection 으로 인정할 최소정확도 지정(confThreshold=0.6, nmsThreshold=0.4)
    classIds, scores, boxes = model.detect(frame, confThreshold=0.6, nmsThreshold=0.4)

    
    # 4.검출된 객체에 박스, 객체명, 정확도 그리기
    for (classId, score, box) in zip(classIds, scores, boxes):
        cv2.rectangle(frame, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]),
                      color=(0, 255, 0), thickness=2)

        text = '%s: %.2f' % (classes[classId[0]], score) # classes[classId](오픈CV:4.5.4이상)
        cv2.putText(frame, text, (box[0], box[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 1,
                    color=(0, 255, 0), thickness=2)
    return frame

# -----------------------
# 3.실시간 카메라 frame 불러오기
# -----------------------
while True:
    ret, frame = cap.read()

    if not ret: 
        print('--Can not read frame---Break!')
        break

    detectAndDisplay(frame)
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == 27:
        break

cap.release()       
cv2.destroyAllWindows()

-------

### [실습] Mediapipe를 얼굴인식

#### Mediapipe 설치하기

In [ ]:
!pip install mediapipe

In [8]:
import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# For webcam input:
cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
                #print(detection)
                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == 27:
            break
            
cap.release()
cv2.destroyAllWindows()

### [실습] Mediapipe를 이용한 Hand Tracking

In [10]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
#    model_complexity=0,    # 오류가 발생하여 주석표시
    max_num_hands=2,  # 인식할 손의 갯수
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()